In [86]:
import tensorflow as tf

## MNIST 데이터 로드하기

In [87]:
from tensorflow.examples.tutorials.mnist import input_data

In [88]:
# 레이블은 정수 형태이지만, 훈련을 위해서 one-hot 인코딩으로 로드
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 퍼셉트론 구축하기

### 1. 입력 데이터와 타깃을 위한 플레이스홀더 정의하기

In [89]:
input_size = 784
no_classes = 10
batch_size = 100
total_batches = 200

# batch size X input_size
x_input = tf.placeholder(tf.float32, shape=[None, input_size])
# batch_size X no_classes. one-hot label
y_input = tf.placeholder(tf.float32, shape=[None, no_classes])

### 2. Fully-connected Layer를 위한 변수 정의하기

In [90]:
# input_size X no_classes (각 픽셀과 클래스 사이의 weight)
weights = tf.Variable(tf.random_normal([input_size, no_classes]))
# no_classes X 1 (클래스 수와 동일)
bias = tf.Variable(tf.random_normal([no_classes]))

In [91]:
# 입력 값에 weight를 부여하고 bias 값 추가
# 각각의 입력 데이터가 어떤 클래스에 해당하는지 예측하고 bias 추가

# 행렬간 곱셈 = batch size X no_classes
logits = tf.matmul(x_input, weights) + bias

In [92]:
# logits과 y_input 비교
softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=y_input, logits=logits)
# cross entropy의 평균으로 loss 계산
loss_operation = tf.reduce_mean(softmax_cross_entropy)
# loss_operation이 minimize되도록 weights, bias 학습
optimizer = tf.train.GradientDescentOptimizer(
    learning_rate=0.5).minimize(loss_operation)

### 3. 데이터로 모델 훈련하기

In [93]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [94]:
for batch_no in range(total_batches):
    mnist_batch = mnist_data.train.next_batch(batch_size)
    _, loss_value = session.run([optimizer, loss_operation], feed_dict={
        x_input: mnist_batch[0],  # 입력 레이블
        y_input: mnist_batch[1]   # 타깃 레이블
    })
    print(loss_value)

12.549507
10.303738
9.886151
10.036405
9.105749
8.109026
7.758338
7.1143303
6.581842
6.6122494
5.4277587
5.9613714
5.493762
4.527266
4.15267
3.9235642
3.8214698
4.7184477
3.6918445
3.940235
3.580413
3.4691718
3.2349524
4.001282
2.4177122
2.518409
2.796557
2.9077854
2.4308178
2.7283382
2.8074083
2.0199962
2.1167533
1.9936433
3.071481
2.5771775
2.463134
2.4444854
2.8249805
2.661185
2.6819897
2.011597
2.351572
2.502522
2.1205957
2.570936
2.6338036
3.2105875
1.8866472
2.0433636
2.4081652
1.7952274
2.3253531
2.083576
1.7523733
1.47138
2.680623
1.7624257
1.6359097
1.6935519
1.5261779
3.147039
1.4984186
2.0957031
1.5685829
1.7229098
1.7797525
1.7845474
1.6160334
1.7892532
1.2114025
0.85204697
1.3429315
1.6069521
1.6100638
1.577338
2.0364125
1.6292624
1.6444706
1.5949218
1.9089807
1.265163
1.9554334
1.649401
1.2783394
0.98577696
1.4205319
1.1842868
1.7267466
1.3236611
1.0429368
1.1885989
1.0147243
1.5921134
1.1755929
1.9423392
1.4069498
1.0841848
1.5959255
1.3499396
1.661831
1.854096
0.8552011

In [95]:
# 모델이 잘 동작하는지 테스트
# tf.argmax: 텐서 내의 지정된 축(0: 열, 1: 행, 2: 면)에서 가장 높은 값의 인덱스 반환

# 정확도 계산
# 1. 가장 큰 값의 인덱스가 예측값
predictions = tf.argmax(logits, 1)
# 2. 예측값과 정답이 일치하는지 비교
correct_predictions = tf.equal(predictions, tf.argmax(y_input, 1))
# 3. 일치 여부를 평균내어 정확도 계산
accuracy_operation = tf.reduce_mean(tf.cast(correct_predictions,
                                            tf.float32))
test_images, test_labels = mnist_data.test.images, mnist_data.test.labels
accuracy_value = session.run(accuracy_operation, feed_dict={
    x_input: test_images,
    y_input: test_labels
})
print('Accuracy : ', accuracy_value)
session.close()

Accuracy :  0.8052
